In [12]:
import pyiwn
import csv

In [13]:
list(map(str, pyiwn.SynsetRelations))

['SynsetRelations.MERO_MEMBER_COLLECTION',
 'SynsetRelations.ABILITY_VERB',
 'SynsetRelations.CAUSATIVE',
 'SynsetRelations.CAPABILITY_VERB',
 'SynsetRelations.MERO_COMPONENT_OBJECT',
 'SynsetRelations.HOLO_PORTION_MASS',
 'SynsetRelations.FUNCTION_VERB',
 'SynsetRelations.HOLO_COMPONENT_OBJECT',
 'SynsetRelations.HYPERNYMY',
 'SynsetRelations.ENTAILMENT',
 'SynsetRelations.ALSO_SEE',
 'SynsetRelations.MERO_FEATURE_ACTIVITY',
 'SynsetRelations.HOLO_PLACE_AREA',
 'SynsetRelations.MODIFIES_VERB',
 'SynsetRelations.ATTRIBUTES',
 'SynsetRelations.MERO_PORTION_MASS',
 'SynsetRelations.MODIFIES_NOUN',
 'SynsetRelations.HOLO_FEATURE_ACTIVITY',
 'SynsetRelations.MERO_STUFF_OBJECT',
 'SynsetRelations.TROPONYMY',
 'SynsetRelations.MERO_PLACE_AREA',
 'SynsetRelations.HOLO_MEMBER_COLLECTION',
 'SynsetRelations.HYPONYMY',
 'SynsetRelations.SIMILAR',
 'SynsetRelations.MERO_POSITION_AREA',
 'SynsetRelations.HOLO_POSITION_AREA',
 'SynsetRelations.HOLO_STUFF_OBJECT']

In [14]:
iwn = pyiwn.IndoWordNet(lang=pyiwn.Language.TELUGU)
synsets = iwn.all_synsets()
pairs = []
# Iterate through each synset
for i, synset in enumerate(synsets[:10]):
    pairs.append((synset.head_word(), synset.gloss()))
    print(f"{pairs[i][0]}\t{pairs[i][1]}")

for i, synset in enumerate(synsets[:20]):
    print(f"-"*90)
    print(f"word: {synset.head_word()}")
    similars = iwn.synset_relation(synset, pyiwn.SynsetRelations.SIMILAR)
    print(f"similars: {similars[:5]}")

    hyponyms = iwn.synset_relation(synset, pyiwn.SynsetRelations.HYPONYMY)
    print(f"hyponyms: {hyponyms[:5]}")

    hypernyms = iwn.synset_relation(synset, pyiwn.SynsetRelations.HYPERNYMY)
    print(f"hypernym: {hypernyms[:5]}")

    entailments = iwn.synset_relation(synset, pyiwn.SynsetRelations.ENTAILMENT)
    print(f"entailment: {entailments[:5]}")

    troponymys = iwn.synset_relation(synset, pyiwn.SynsetRelations.TROPONYMY)
    print(f"troponymy: {troponymys[:5]}")

    # hypernyms = iwn.synset_relation(synset, pyiwn.SynsetRelations.HYPERNYMY)
    # print(f"hypernym: {hypernyms[:5]}")

2024-10-07:22:26:52,14 INFO     [iwn.py:43] Loading telugu language synsets...


వెలివేయబడిన	ఇంకా జన్మించని వాడు.
అశుభం	మంచి లేదా మంగళకరం కానిది
ప్రవేశంలేని	లోనికిరాని
పవిత్రస్థానం	ఆ స్థలం పవిత్రమైనదిగా భావిస్తారు
శివాలయం	శివుని ఆరాధించుటకు శివుని విగ్రహం ప్రతిష్టించిన మందిరం
అపవిత్ర_స్థానం	ఏదైతే పవిత్ర స్థలం కాదో
వచ్చిన	ఎవరైతే వస్తారో
జన్మించిన	తల్లి గర్భం నుండి కొత్తగా భూమిపైకి రావడం
నీతికిసంబంధించిన_పని	అవినీతికి సంబంధించినది కాదు
పండించిన	ఏదైన ఉత్పత్తి చేసిన.
------------------------------------------------------------------------------------------
word: వెలివేయబడిన
similars: []
hyponyms: []
hypernym: []
entailment: []
troponymy: []
------------------------------------------------------------------------------------------
word: అశుభం
similars: []
hyponyms: []
hypernym: []
entailment: []
troponymy: []
------------------------------------------------------------------------------------------
word: ప్రవేశంలేని
similars: []
hyponyms: []
hypernym: []
entailment: []
troponymy: []
----------------------------------------------------------------------------------------

In [25]:
import pyiwn
import csv

# Function to read the large corpus and create a set of words
def load_corpus_as_set(corpus_file):
    word_set = set()
    with open(corpus_file, 'r', encoding='utf-8') as file:
        for line in file:
            # Assuming each line contains space-separated words
            words = line.strip().split()
            word_set.update(words)
    return word_set

# Function to extract WordNet word-definition pairs
def extract_word_definition_pairs():
    synsets = iwn.all_synsets()
    pairs = []
    # Iterate through each synset
    for synset in synsets:
        pairs.append((synset.head_word(), synset.gloss(), synset))

    return pairs


def create_tsv_with_presence_check(wordnet_data, corpus_file, output_file):
    corpus_word_set = load_corpus_as_set(corpus_file)

    word_def_pairs = extract_word_definition_pairs()

    with open(output_file, 'w', newline='', encoding='utf-8') as tsv_file:
        tsv_writer = csv.writer(tsv_file, delimiter='\t')
        tsv_writer.writerow(['index', 'word_a', 'word_b', 'label'])

        for index, (word, definition, synset) in enumerate(word_def_pairs):
            is_present = 1 if word in corpus_word_set else 0

            if is_present == 0 and "_" not in word and " " not in word:
                similars = iwn.synset_relation(synset, pyiwn.SynsetRelations.SIMILAR)
                hypernyms = iwn.synset_relation(synset, pyiwn.SynsetRelations.HYPERNYMY)
                hyponyms = iwn.synset_relation(synset, pyiwn.SynsetRelations.HYPONYMY)
                similar_words_list = []
                for similar in similars:
                    similar_words_list.append(similar.head_word())
                for hypernym in hypernyms:
                    similar_words_list.append(hypernym.head_word())
                for hyponym in hyponyms:
                    similar_words_list.append(hyponym.head_word())

                if len(similar_words_list) > 0:
                    tsv_writer.writerow([index, synset.head_word(), similar_words_list, 1])

# Example usage
LANG = "te"
LANG_FULL = "telugu"
corpus_file = f'/media/saketh/New Volume/NAACL 2025/Datasets/{LANG}/{LANG}_10M_splits.txt'  # Your 4GB large corpus file
output_file = f'{LANG_FULL}/WaW_{LANG}_lemma.tsv'

iwn = pyiwn.IndoWordNet(lang=pyiwn.Language.TELUGU)

# Create the TSV file
create_tsv_with_presence_check(extract_word_definition_pairs(), corpus_file, output_file)

print(f"TSV file with word presence check created: {output_file}")


2024-10-07:23:21:18,47 INFO     [iwn.py:43] Loading telugu language synsets...


TSV file with word presence check created: telugu/WaW_te_lemma.tsv


# Create Negative Samples

In [5]:
import random
from collections import defaultdict
import pyiwn
import csv

# Function to read the large corpus and create a set of words
def load_corpus_as_set(corpus_file):
    word_set = set()
    with open(corpus_file, 'r', encoding='utf-8') as file:
        for line in file:
            # Assuming each line contains space-separated words
            words = line.strip().split()
            word_set.update(words)
    return word_set

# Function to extract WordNet word-definition pairs
def extract_word_definition_pairs():
    synsets = iwn.all_synsets()
    pairs = []
    # Iterate through each synset
    for synset in synsets:
        pairs.append((synset.head_word(), synset.gloss(), synset))

    return pairs

def generate_negative_samples(word_def_pairs, positive_pairs_file, output_file):
    # Load existing positive pairs
    positive_pairs = set()
    word_pos_neighbors = defaultdict(set)
    all_words = set()
    
    with open(positive_pairs_file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter='\t')
        next(reader)  # Skip header
        for row in reader:
            word_a = row[1]
            similar_words = eval(row[2])  # Convert string representation of list back to list
            
            # Store positive pairs and build neighborhood dictionary
            for word_b in similar_words:
                positive_pairs.add((word_a, word_b))
                positive_pairs.add((word_b, word_a))  # Add both directions
                word_pos_neighbors[word_a].update(similar_words)
                word_pos_neighbors[word_b].update([word_a])
            
            all_words.add(word_a)
            all_words.update(similar_words)
    
    # Convert to list for random sampling
    all_words = list(all_words)
    
    def is_valid_negative_pair(word1, word2):
        # Check if this pair or its reverse is not already positive
        if (word1, word2) in positive_pairs or (word2, word1) in positive_pairs:
            return False
            
        # Check if words share any neighbors (indirect relationship)
        if word_pos_neighbors[word1] & word_pos_neighbors[word2]:
            return False
            
        # Additional WordNet-based checks
        word1_synsets = iwn.synsets(word1)
        word2_synsets = iwn.synsets(word2)
        
        for s1 in word1_synsets:
            for s2 in word2_synsets:
                # Check if they share any hypernym
                hyper1 = set(iwn.synset_relation(s1, pyiwn.SynsetRelations.HYPERNYMY))
                hyper2 = set(iwn.synset_relation(s2, pyiwn.SynsetRelations.HYPERNYMY))
                if hyper1 & hyper2:
                    return False
                
                # Check for other semantic relationships
                if s2 in iwn.synset_relation(s1, pyiwn.SynsetRelations.ENTAILMENT):
                    return False
        
        return True

    # Generate negative samples
    negative_pairs = set()
    required_negative_samples = len(positive_pairs)
    
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerow(['index', 'word_a', 'word_b', 'label'])
        
        attempt_count = 0
        max_attempts = required_negative_samples * 10  # Limit attempts to avoid infinite loop
        
        while len(negative_pairs) < required_negative_samples and attempt_count < max_attempts:
            word1 = random.choice(all_words)
            word2 = random.choice(all_words)
            
            if word1 != word2 and is_valid_negative_pair(word1, word2):
                negative_pairs.add((word1, word2))
                writer.writerow([len(negative_pairs), word1, word2, 0])
            
            attempt_count += 1
            
        if len(negative_pairs) < required_negative_samples:
            print(f"Warning: Could only generate {len(negative_pairs)} negative samples out of {required_negative_samples} requested")
        
    return negative_pairs

# Usage
LANG = "te"
LANG_FULL = "telugu"
corpus_file = f'/media/saketh/New Volume/NAACL 2025/Datasets/{LANG}/{LANG}_10M_splits.txt'  # Your 4GB large corpus file
output_file = f'{LANG_FULL}/WaW_{LANG}_copy.tsv'
iwn = pyiwn.IndoWordNet(lang=pyiwn.Language.TELUGU)
word_def_pairs = extract_word_definition_pairs()
negative_samples_file = f'{LANG_FULL}/WaW_{LANG}_lemma_negative.tsv'
negative_pairs = generate_negative_samples(word_def_pairs, output_file, negative_samples_file)

2024-10-25:21:22:21,786 INFO     [iwn.py:43] Loading telugu language synsets...


# Create balanced test data

In [10]:
import random
import csv
from collections import defaultdict
import pandas as pd

def create_balanced_test_data(train_positive_file, train_negative_file, test_size, output_file):
    """
    Creates a balanced test dataset with equal positive and negative samples,
    ensuring no overlap with training data.
    """
    # Load all training data to avoid test-train overlap
    train_pairs = set()
    train_words = set()
    
    # Load positive training pairs
    with open(train_positive_file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter='\t')
        next(reader)  # Skip header
        for row in reader:
            word_a = row[1]
            similar_words = eval(row[2])
            for word_b in similar_words:
                train_pairs.add((word_a, word_b))
                train_pairs.add((word_b, word_a))
                train_words.add(word_a)
                train_words.add(word_b)

    # Load negative training pairs
    with open(train_negative_file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter='\t')
        next(reader)  # Skip header
        for row in reader:
            word_a, word_b = row[1], row[2]
            train_pairs.add((word_a, word_b))
            train_pairs.add((word_b, word_a))
            train_words.add(word_a)
            train_words.add(word_b)

    # Get all synsets and their relations for positive sampling
    all_synsets = iwn.all_synsets()
    synset_relations = defaultdict(list)
    
    for synset in all_synsets:
        head_word = synset.head_word()
        if head_word not in train_words:  # Only consider words not in training
            # Get related synsets
            similars = iwn.synset_relation(synset, pyiwn.SynsetRelations.SIMILAR)
            hypernyms = iwn.synset_relation(synset, pyiwn.SynsetRelations.HYPERNYMY)
            hyponyms = iwn.synset_relation(synset, pyiwn.SynsetRelations.HYPONYMY)
            
            for related_synset in similars + hypernyms + hyponyms:
                related_word = related_synset.head_word()
                if related_word not in train_words:
                    synset_relations[head_word].append(related_word)

    def generate_positive_samples(needed_samples):
        positive_pairs = []
        available_words = list(synset_relations.keys())
        
        while len(positive_pairs) < needed_samples and available_words:
            word_a = random.choice(available_words)
            if synset_relations[word_a]:
                word_b = random.choice(synset_relations[word_a])
                if (word_a, word_b) not in train_pairs and (word_b, word_a) not in train_pairs:
                    positive_pairs.append((word_a, word_b, 1))
                    # Remove used words to ensure diverse sampling
                    if len(synset_relations[word_a]) == 1:
                        available_words.remove(word_a)
        
        return positive_pairs

    def generate_negative_samples(needed_samples, positive_test_pairs):
        negative_pairs = []
        all_test_words = set([word for pair in positive_test_pairs for word in pair[:2]])
        all_test_words = list(all_test_words)
        
        def is_valid_negative_pair(word1, word2):
            # Check if not in training pairs
            if (word1, word2) in train_pairs or (word2, word1) in train_pairs:
                return False
                
            # Check if not in positive test pairs
            if (word1, word2, 1) in positive_test_pairs or (word2, word1, 1) in positive_test_pairs:
                return False
                
            # Check WordNet relationships
            word1_synsets = iwn.synsets(word1)
            word2_synsets = iwn.synsets(word2)
            
            for s1 in word1_synsets:
                for s2 in word2_synsets:
                    # Check various semantic relationships
                    if (s2 in iwn.synset_relation(s1, pyiwn.SynsetRelations.SIMILAR) or
                        s2 in iwn.synset_relation(s1, pyiwn.SynsetRelations.HYPERNYMY) or
                        s2 in iwn.synset_relation(s1, pyiwn.SynsetRelations.HYPONYMY)):
                        return False
            
            return True

        attempts = 0
        max_attempts = needed_samples * 10
        
        while len(negative_pairs) < needed_samples and attempts < max_attempts:
            word1 = random.choice(all_test_words)
            word2 = random.choice(all_test_words)
            
            if word1 != word2 and is_valid_negative_pair(word1, word2):
                negative_pairs.append((word1, word2, 0))
            
            attempts += 1
        
        return negative_pairs

    # Generate balanced test data
    samples_per_class = test_size // 2
    positive_samples = generate_positive_samples(samples_per_class)
    negative_samples = generate_negative_samples(samples_per_class, positive_samples)
    
    # Combine and shuffle test data
    test_data = positive_samples + negative_samples
    random.shuffle(test_data)
    
    # Save test data
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerow(['index', 'word_a', 'word_b', 'label'])
        for idx, (word_a, word_b, label) in enumerate(test_data):
            writer.writerow([idx, word_a, word_b, label])
    
    # Generate statistics
    stats = {
        'total_samples': len(test_data),
        'positive_samples': len(positive_samples),
        'negative_samples': len(negative_samples),
        'unique_words': len(set(word for pair in test_data for word in pair[:2])),
        'avg_word_length': sum(len(word) for pair in test_data for word in pair[:2]) / (len(test_data) * 2)
    }
    
    return test_data, stats

# Usage example
test_size = 10000  # Specify desired test set size (will be split equally between positive and negative)
test_file = f'{LANG_FULL}/WaW_{LANG}_test.tsv'

test_data, stats = create_balanced_test_data(
    train_positive_file=f'{LANG_FULL}/WaW_{LANG}_positive.tsv',
    train_negative_file=f'{LANG_FULL}/WaW_{LANG}_negative.tsv',
    test_size=test_size,
    output_file=test_file
)

print("Test Data Statistics:")
for key, value in stats.items():
    print(f"{key}: {value}")

Test Data Statistics:
total_samples: 10000
positive_samples: 5000
negative_samples: 5000
unique_words: 1610
avg_word_length: 8.2207
